In [1]:
from torch import cuda, bfloat16
import transformers
from transformers import (
    pipeline,
    logging,
)
import pandas as pd
import os

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

hf_token = os.environ.get('hf_token')

# Need auth token for these
hf_auth = hf_token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [4]:
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2056)
result = pipe(f"Using pybgpstream identify prefixes associated with ASN AS4766 that show high update frequency (announcements and withdrawals) within the period oct 28 13:00 to oct 28 13:15, 2024. Summarize the prefixes, update counts, and provide an assessment of their stability.")
print(result[0]['generated_text'])

Using pybgpstream identify prefixes associated with ASN AS4766 that show high update frequency (announcements and withdrawals) within the period oct 28 13:00 to oct 28 13:15, 2024. Summarize the prefixes, update counts, and provide an assessment of their stability. 

```python
import pybgpstream

# Create a BGP Stream reader
reader = pybgpstream.BGPStreamReader(
    'bgpstream://localhost:4739?start_time=2024-10-28T13:00:00&end_time=2024-10-28T13:15:00&as=4766&prefixes=',
    'json'
)

# Initialize a dictionary to store the prefixes and their update counts
prefixes = {}

# Iterate over the BGP stream
for record in reader:
    # Get the prefix and the update type (announcement or withdrawal)
    prefix = record.prefix
    update_type = record.update_type

    # If the prefix is not in the dictionary, add it with an update count of 1
    if prefix not in prefixes:
        prefixes[prefix] = {'announcements': 1, 'withdrawals': 1}
    # If the prefix is already in the dictionary, increment

In [5]:
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2056)
result = pipe(f"Using pybgpstream summarize the count of BGP update messages (announcements and withdrawals) for ASN AS4766 between oct 28 13:00 and oct 28 13:15, 2024. Provide total counts as well as breakdowns by 5-minute intervals.")
print(result[0]['generated_text'])

Using pybgpstream summarize the count of BGP update messages (announcements and withdrawals) for ASN AS4766 between oct 28 13:00 and oct 28 13:15, 2024. Provide total counts as well as breakdowns by 5-minute intervals. 

```python
import pybgpstream

# Define the time interval
start_time = '2024-10-28 13:00:00'
end_time = '2024-10-28 13:15:00'

# Define the ASN
asn = '4766'

# Create a BGP stream reader
reader = pybgpstream.BGPStreamReader()

# Define the query
query = pybgpstream.Query()
query.set_asn(asn)
query.set_start_time(start_time)
query.set_end_time(end_time)

# Add the query to the reader
reader.add_query(query)

# Read the stream
reader.read_next()

# Initialize counters
total_announcements = 0
total_withdrawals = 0

# Initialize a dictionary to store 5-minute interval counts
interval_counts = {}

# Loop through the stream
while reader.read_next():
    record = reader.get_record()
    if record.get_type() == 'update':
        total_announcements += 1
        interval_counts[